<a href="https://colab.research.google.com/github/dhikaainul/Pertemuan-12/blob/main/js11_01_teori_parameter_tunning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Parameter Tunning

Pada jobsheet ini kita akan belajar bagaimana cara mengoptimisasi (*tunning*) algoritma machine learning untuk mendapatkan model yang lebih baik. Namun sebelum itu, kita akan belajar tentang konsep *splitting* data.

## Membagi Data (*Splitting Data*)

### Data *training* (latih), *validation* (validasi), testing (*pengujian*)

Pada pembelajaran yang lalu, kita sudah mengetahui apa itu data training dan apa itu data testing. Selain itu, kita juga telah mempraktikkan bagaimana cara melakukan pembagian (*splitting*) antara data training dan data testing menggunakan berbagai rasio. Pertanyaannya adalah, apa itu data validasi (validation data)? Mengapa kita memerlukan data validasi pada saat mengembangkan sebuah model? Untuk menjawab hal tersebut, kita kebali lagi ke definisi masing-masing terlebih dahulu.

> **Data training**, adalah data (dataset) yang kita gunakan untuk melatih algoritma machine learning. Model machine learning akan melihat dan belajar dari data ini.

> **Data testing**, adalah sebagian data dari seluruh data (sampel data) yang digunakan untuk mengevaluasi model machine learning yang **dihasilkan** dari proses training.

> **Data validasi**, adalah data sebagian dari seluruh data (sampel data) yang digunakan untuk mengevaluasi model machine learning **pada saat optimisasi atau penyetelan hyperparameter**

Apakah kita benar-benar membutuhkan data validasi?

Jawabannya adalah tergantung. Jika kita tidak ingin melakukan optimisasi hyperparameter, atau jumlah hyperparameter sudah algoritma terlalu banyak sehingga sulit untuk di optimisasi, maka kita tidak perlu membuat data validasi. **Tujuan dari data validasi adalah membantu kita untuk menghasilkan evaluasi yang tidak bias dari pengujian model dengan menggunakan data testing.**

#### Rasio data training, validasi, dan testing

Setelah kita mengetahui definisi data training, validasi, dan testing, mungkin kita akan bertanya terakit seberapa seberapa besar data yang harus kita gunakan untuk masing-masing jenis data. Terdapat beberapa pendapat terkait dengan ini. Jika model memiliki sedikit hyperparamter, maka kita dapat mengurangi jumlah dari data validasi. Sebaliknya, jika model memiliki banyak hyperparameter, maka kita dapat meningkatkan jumlah dari data validasi. Gambar dibawah ini merupakan ilustrasi proporsi data training, validasi, dan testing.

![https://towardsdatascience.com/train-validation-and-test-sets-72cb40cba9e7](assets/data_ratio.png)

Akan tetapi, menurut Lois Owen dalam bukunya yang berjudul Hyperparameter Tunning with Python, rasio yang umum digunakan adalah 8:1:1 atau 9:0.50.5 untuk data training, validasi dan testing secara berurutan.

### Random split dan stratified split

Pemisahan antar jenis data untuk kebutuhan pembuatan model sudah sering ktia lakukan pada pembelajaran sebelumnya. Pada modul-modul yang lalu, kita menggunakan ***random split*** untuk melakukan hal tersebut. Akan tetapi, terdapat teknik lain yang dapat kita gunakan untuk melakukan *splitting* data, yaitu *stratified split* atau *stratified random split*. Sedikit berbeda dengan random split, dimana kita langsung memilih secara acak data yang akan kita gunakan sebagai data training, valiasi, maupun testing, pada *stratified split* kita mengenal strata pada prosesnya.

Apa maksud dari strata (*stratified*). Strata dalam konteks ini adalah kelompok, atau kita dapat menganologikannya sebagai label. *Stratified split* akan membagi data kedalam porsi training, validasi, dan testing sesuai dengan proporsi setiap label atau kelas. Hal ini menyebabkan jumlah tiap label akan memiliki rasio yang sama. Pada ilmus statistika, teknik ini juga masuk dalam teknik sampling. Gambar dibawah ini merupakan ilustrasi dari *stratified sampling* yang digunakan pada ilmu statistika.

![stratified random sampling](assets/stratified_sampling.png)
![proportioned stratified random sampling](assets/proportioned_stratified_sampling.png)

### Implementasi pembagian data

Pada contoh ini, kita akan mencoba membagi data keadalam data trainig, validasi, dan testing dengan menggunakan scikit-learn.

In [1]:
# load data
import pandas as pd

df = pd.read_csv('wbc_2.csv')
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [2]:
# Split data
from sklearn.model_selection import train_test_split

# Split data training dan dan lainnya
# data lainnya, akan kita split lagi menjadi validasi dan testing.
# Rasio yang akan kita gunakan adalah 8:1:1
df_train, df_unseen = train_test_split(df, test_size=0.2, random_state=0)

# Split lagi antara validasi dan testing
df_val, df_test = train_test_split(df_unseen, test_size=0.5, random_state=0)

# Cek masing-masing ukuran data

print(f'Jumlah data asli: {df.shape[0]}')
print(f'Jumlah data train: {df_train.shape[0]}')
print(f'Jumlah data val: {df_val.shape[0]}')
print(f'Jumlah data test: {df_test.shape[0]}')

# Cek rasio tiap label
print('=========')
print(f'Jumlah label data asli:\n{df.diagnosis.value_counts()}')
print(f'Jumlah label data train:\n{df_train.diagnosis.value_counts()}')
print(f'Jumlah label data val:\n{df_val.diagnosis.value_counts()}')
print(f'Jumlah label data test:\n{df_test.diagnosis.value_counts()}')

Jumlah data asli: 569
Jumlah data train: 455
Jumlah data val: 57
Jumlah data test: 57
Jumlah label data asli:
B    357
M    212
Name: diagnosis, dtype: int64
Jumlah label data train:
B    290
M    165
Name: diagnosis, dtype: int64
Jumlah label data val:
B    35
M    22
Name: diagnosis, dtype: int64
Jumlah label data test:
B    32
M    25
Name: diagnosis, dtype: int64


#### Stratified Split

Bagaimana jika kita ingin melakukan stratified random split? Kita cukup memasukkan parameter ***stratify*** dengan informasi label/kelas pada fungsi ***train_test_split***

In [3]:
# Split data
from sklearn.model_selection import train_test_split

# Split data training dan dan lainnya
# data lainnya, akan kita split lagi menjadi validasi dan testing.
# Rasio yang akan kita gunakan adalah 8:1:1
df_train, df_unseen = train_test_split(df, test_size=0.2, random_state=0, stratify=df['diagnosis'])

# Split lagi antara validasi dan testing
df_val, df_test = train_test_split(df_unseen, test_size=0.5, random_state=0, stratify=df_unseen['diagnosis'])

# Cek masing-masing ukuran data

print(f'Jumlah label data asli:\n{df.diagnosis.value_counts()}')
print(f'Jumlah label data train:\n{df_train.diagnosis.value_counts()}')
print(f'Jumlah label data val:\n{df_val.diagnosis.value_counts()}')
print(f'Jumlah label data test:\n{df_test.diagnosis.value_counts()}')

Jumlah label data asli:
B    357
M    212
Name: diagnosis, dtype: int64
Jumlah label data train:
B    285
M    170
Name: diagnosis, dtype: int64
Jumlah label data val:
B    36
M    21
Name: diagnosis, dtype: int64
Jumlah label data test:
B    36
M    21
Name: diagnosis, dtype: int64


### Pembagian data dengan *k-fold cross validation*

Setelah kita memahami bagaimana cara melakukan spliting data training, validasi, dan testing, pada bagian ini kita akan belajar terkait dengan teknik lain dalam melakukan *splitting* data, yaitu *k-fold cross validation*. Apa itu *k-fold cross validation*? *k-fold cross validation* adalah teknik pada machine leanring untuk mengevaluasi model dengan cara melakukan evaluasi berganda (*multiple evaluation*). Evaluasi berganda dilakukan dengan cara membagi data menjadi data training dan validasi, yang diistilahkan sebagai ***fold*** sedemiakan sehingga setiap data pernah menjadi data training ataupun data testing. Hasil testing pada model splitting ini adalah rata-rata hasil untuk setiap *fold*. Besaran spliting data akan ditentukan oleh nilai *k*-nya. Sebagai contoh, jika kita menggunakan nilai $k=4$, maka data akan dibagi menjadi 4 bagian. Salah satu dari bagian tersebut akan menjadi data validasi. Kemudian, untuk setiap iterasi, data validasi akan diganti sesuai dengan jumlah *fold*. Untuk memudahkan pemahaman, perhatikan gambar dibawah ini.

![kfold cross validation](assets/kfold_cross_validation.png)

Berdasarkan gambar diatas, ktia dapat mengetahui bahwa, untuk setiap *fold* pada *k-fold cross validation* akan digunakan baik sebagai data training ataupun testing. Sebagai tambahan, **konsep random sampling dan stratified random sampling juga dapat diterapkan pada teknik ini**.

#### Implementasi *k-fold cross validation* (random)

Pada contoh ini, kita akan menggunakan data WBC untuk mempraktikkan teknik *k-fold cross validation*

In [4]:
# Cek df apakah tetap sama

display(df.head())
print(f'Dimensi data WBC: {df.shape}')

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


Dimensi data WBC: (569, 32)


In [5]:
# Implementasi k-fold cross validation (random) dengan training dan testing saja
from sklearn.model_selection import KFold

# inisiasi obyek kfold
kf = KFold(n_splits=4)
print(f'Jumlah fold: {kf.get_n_splits()}')
print(f'Obyek KFold: {kf}')

# Lakukan splitting dengan KFold
kf_split = kf.split(df)
print(f'Jumlah data df: {df.shape[0]}')

# cek index data tiap fold
for train_index, test_index in kf_split:
    print(f'Index train: {train_index} | Index test: {test_index}')

Jumlah fold: 4
Obyek KFold: KFold(n_splits=4, random_state=None, shuffle=False)
Jumlah data df: 569
Index train: [143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160
 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178
 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196
 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214
 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232
 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250
 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268
 269 270 271 272 273 274 275 276 277 278 279 280 281 282 283 284 285 286
 287 288 289 290 291 292 293 294 295 296 297 298 299 300 301 302 303 304
 305 306 307 308 309 310 311 312 313 314 315 316 317 318 319 320 321 322
 323 324 325 326 327 328 329 330 331 332 333 334 335 336 337 338 339 340
 341 342 343 344 345 346 347 348 349 350 351 352 353 354 355 356 357 358
 359 360 36

In [6]:
# Implementasi k-fold cross validation (random) dengan training, validation, dan testing data
from sklearn.model_selection import train_test_split, KFold

# Split dulu antara data training dan testing dengan train_test_split
# Rasio 8:2 untuk training dan testing
df_train, df_test = train_test_split(df, test_size=0.2, random_state=0)

# inisiasi obyek kfold
kf2 = KFold(n_splits=4)
print(f'Jumlah fold: {kf2.get_n_splits()}')
print(f'Obyek KFold: {kf2}')

# Lakukan splitting dengan KFold untuk data df_training
# Dengan acara ini, kita masih memiliki data testing untuk keperluan pengujian model
# namun tetap dapat melakukan evaluasi dengan menggunakan data validasi
kf2_split = kf2.split(df_train)
print(f'Jumlah data df_train: {df_train.shape[0]}')

# cek index data tiap fold
for train_index, test_index in kf2_split:
    print(f'Index train: {train_index} | Index test: {test_index}')

Jumlah fold: 4
Obyek KFold: KFold(n_splits=4, random_state=None, shuffle=False)
Jumlah data df_train: 455
Index train: [114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131
 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149
 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167
 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185
 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203
 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221
 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239
 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257
 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275
 276 277 278 279 280 281 282 283 284 285 286 287 288 289 290 291 292 293
 294 295 296 297 298 299 300 301 302 303 304 305 306 307 308 309 310 311
 312 313 314 315 316 317 318 319 320 321 322 323 324 325 326 327 328 329
 330 

#### Implementasi *k-fold cross validation* (stratified)

Selanjutnya, kita akan mencoba melakukan *k-fold cross validation* dengan menggunakan konsep stratified. 

In [7]:
# Implementasi k-fold cross validation stratified hanya dengan data training dan testing
from sklearn.model_selection import StratifiedKFold

# inisiasi obyek StratifiedKFold
skf = StratifiedKFold(n_splits=4)
print(f'Jumlah fold: {skf.get_n_splits()}')
print(f'Obyek SKFold: {skf}')

# Lakukan splitting dengan SKFold
# parameter kedua dari split adalah labelnya
skf_split = skf.split(df, df['diagnosis'])
print(f'Jumlah data df: {df.shape[0]}')

# cek index data tiap fold
for train_index, test_index in skf_split:
    print(f'Index train: {train_index} | Index test: {test_index}')

Jumlah fold: 4
Obyek SKFold: StratifiedKFold(n_splits=4, random_state=None, shuffle=False)
Jumlah data df: 569
Index train: [ 75  77  78  82  83  85  86  87  91  94  95  99 100 105 108 117 118 119
 121 122 126 127 129 131 132 134 135 138 141 146 156 161 162 164 167 168
 171 172 177 180 181 182 184 186 187 188 189 190 191 192 193 194 195 196
 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214
 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232
 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250
 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268
 269 270 271 272 273 274 275 276 277 278 279 280 281 282 283 284 285 286
 287 288 289 290 291 292 293 294 295 296 297 298 299 300 301 302 303 304
 305 306 307 308 309 310 311 312 313 314 315 316 317 318 319 320 321 322
 323 324 325 326 327 328 329 330 331 332 333 334 335 336 337 338 339 340
 341 342 343 344 345 346 347 348 349 350 351 352 353 354 355 356 357 358


In [8]:
# Implementasi k-fold cross validation stratified dengan data training, validation, dan testing
from sklearn.model_selection import train_test_split, StratifiedKFold

# Split train test
# Rasio 8:2
df_train2, df_test2 = train_test_split(df, test_size=0.2, random_state=0)

# inisiasi obyek StratifiedKFold
# dengan data df_train2
skf2 = StratifiedKFold(n_splits=4)
print(f'Jumlah fold: {skf2.get_n_splits()}')
print(f'Obyek SKFold: {skf2}')

# Lakukan splitting dengan SKFold
# parameter kedua dari split adalah labelnya
# Split train menjadi train dan validasi
skf2_split = skf.split(df_train2, df_train2['diagnosis'])
print(f'Jumlah data df: {df_train.shape[0]}')

# cek index data tiap fold
for train_index, test_index in skf2_split:
    print(f'Index train: {train_index} | Index test: {test_index}')

Jumlah fold: 4
Obyek SKFold: StratifiedKFold(n_splits=4, random_state=None, shuffle=False)
Jumlah data df: 455
Index train: [107 109 110 111 112 114 116 117 118 120 121 122 124 125 128 129 130 131
 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149
 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167
 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185
 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203
 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221
 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239
 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257
 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275
 276 277 278 279 280 281 282 283 284 285 286 287 288 289 290 291 292 293
 294 295 296 297 298 299 300 301 302 303 304 305 306 307 308 309 310 311
 312 313 314 315 316 317 318 319 320 321 322 323 324 325 326 327 328 329


### Pembagian data dengan repeated k-fold cross validation

Pada bagian ini kita akan melakukan satu teknik cross validation lagi yaitu repeated k-fold cross validation. Perbedaan utama dari teknik ini adalah kita akan melakukan perulangan terhadap k-fold cross validation sebanyak $N$ kali sehingga menghasilkan data fold yang berbeda untuk setiap pengulangan. Hasil evaluasi dari teknik ini adalah rata-rata performa setiap perulangan dan setiap *fold*. Dengan cara ini, kita dapat meningkatkan kepercayaan terhadap model yang dihasilkan. 

Lantas mengapa kita menggunakan *repeated k-fold*? Kenapa tidak kita tingkatan saja nilai $k$-nya?

> Meningkatkan nilai $k$ memang benar akan menurunkan nilai bias, akan tetapi disaat yang bersamaan akan meningkatkan nilai variasi data (karena jumlah $k$ yang banyak) terutama pada jumlah data yang kecil. Dengan menggunakan *repeated k-fold* kita dapat meminimalisir hal ini, tentu saja dengan *trade-off* yang layak, yaitu nilai komputasi yang meningkat.

#### Implementasi repeated k-fold cross validation

Kita akan mencoba menerapkan repeated k-fold dengan 2 skenario kembali, yaitu dengan validasi dan tidak. Selain itu, teknik ini juga bisa digunakan pada kondisi *stratified*. Penerapan repeated k-fold akan kita lakukan dengan menggunakan scikit-learn

In [9]:
# Implementasi repeated k-fold cross validation hanya dengan data training dan testing
from sklearn.model_selection import RepeatedKFold

# inisiasi obyek RepeatedKFold
rkf = RepeatedKFold(n_splits=4, n_repeats=3) # n_repeats digunakan untuk mendefinisikan berapa kali ulangan
print(f'Jumlah fold: {rkf.get_n_splits()}')
print(f'Obyek SKFold: {rkf}')

# Lakukan splitting dengan RKFold
# parameter kedua dari split adalah labelnya
rkf_split = rkf.split(df, df['diagnosis'])
print(f'Jumlah data df: {df.shape[0]}')

# cek index data tiap fold
for train_index, test_index in rkf_split:
    print(f'Index train: {train_index} | Index test: {test_index}')

Jumlah fold: 12
Obyek SKFold: RepeatedKFold(n_repeats=3, n_splits=4, random_state=None)
Jumlah data df: 569
Index train: [  2   3   4   5   7   9  10  11  12  13  14  16  18  19  20  22  23  24
  26  31  33  35  36  37  38  40  41  42  44  45  46  47  49  50  51  54
  55  56  58  60  61  62  64  65  67  68  69  70  71  72  73  74  75  76
  77  78  79  80  82  83  84  85  86  87  89  90  91  92  93  96  97  98
  99 100 102 106 107 109 110 112 114 115 116 117 118 120 121 122 123 124
 125 126 127 128 131 133 135 139 140 141 142 143 144 145 146 147 148 149
 151 153 154 155 156 157 158 159 160 161 162 163 164 166 168 169 170 171
 172 173 175 176 177 178 181 182 183 185 186 189 190 191 192 194 195 196
 197 198 199 200 201 203 204 205 206 208 209 212 213 214 215 216 218 221
 223 224 226 227 228 230 231 232 233 234 235 237 238 239 240 241 242 243
 245 246 247 248 249 250 251 252 254 255 257 258 259 261 262 263 265 267
 268 269 270 271 272 274 275 276 277 278 279 280 281 282 283 284 285 288
 28

In [10]:
# Implementasi repeated k-fold cross validation data training, validasi, dan testing dengan stratified
from sklearn.model_selection import train_test_split, RepeatedKFold

# Split train test dengan stratified
df_train3, df_test3 = train_test_split(df, test_size=0.8, random_state=0, stratify=df['diagnosis'])


# inisiasi obyek RepeatedKFold
rkf2 = RepeatedKFold(n_splits=4, n_repeats=3) # n_repeats digunakan untuk mendefinisikan berapa kali ulangan
print(f'Jumlah fold: {rkf2.get_n_splits()}')
print(f'Obyek SKFold: {rkf2}')

# Lakukan splitting dengan RKFold
# parameter kedua dari split adalah labelnya
rkf_split2 = rkf2.split(df_train3, df_train3['diagnosis'])
print(f'Jumlah data df: {df_train3.shape[0]}')

# cek index data tiap fold
for train_index, test_index in rkf_split2:
    print(f'Index train: {train_index} | Index test: {test_index}')

Jumlah fold: 12
Obyek SKFold: RepeatedKFold(n_repeats=3, n_splits=4, random_state=None)
Jumlah data df: 113
Index train: [  0   2   3   7   9  10  11  13  14  15  16  17  18  19  20  21  22  23
  25  26  27  29  30  32  34  36  37  38  39  40  41  42  43  45  46  48
  50  51  52  53  54  56  57  58  59  60  61  62  65  66  67  68  69  70
  71  72  73  74  75  76  77  79  80  82  83  86  88  89  90  91  93  94
  95  97 100 102 104 105 106 107 108 109 110 112] | Index test: [  1   4   5   6   8  12  24  28  31  33  35  44  47  49  55  63  64  78
  81  84  85  87  92  96  98  99 101 103 111]
Index train: [  1   2   4   5   6   8  10  11  12  14  15  16  17  18  19  22  23  24
  26  27  28  29  30  31  33  35  37  38  39  41  44  45  46  47  49  51
  52  55  56  57  58  60  61  62  63  64  66  68  70  71  72  74  75  76
  77  78  79  81  82  84  85  86  87  88  89  90  91  92  94  95  96  97
  98  99 100 101 102 103 104 106 107 108 109 111 112] | Index test: [  0   3   7   9  13  20  21  2

## Hyperparameter Tunning / Optimization

Setelah kita mengetahui berbagai teknik *splitting* data, kita akan masuk kedalam pembahasan *hyperparameter tunning* atau dikenal juga sebagai *hyperparameter optimization*. Apa itu *hyperparameter tunning*? Sebelum itu, kita harus mengetahui terlebih dahulu apa itu *hyperparamter* pada konteks machine learning, khususnya algoritma machine learning.

> *Hyperparameter* adalah parameter yang digunakan oleh algortima machine learning untuk membentuk sebuah model machine learning. Nilai parameter telah ditentukan sebelum proses "pembelajaran" berlangsung.

Dikarenakan setiap algoritma machine learning memiliki parameternya masing-masing (sebagai contoh *decision tree* dengan nilai maksimal split, jumlah node dan leaf, ataupun kriteria split), maka setiap proses tunning akan bergantung pada algoritma atau model machine learning apa yang digunakan.

Lalu apa yang dimaksud dengan *hyperparameter tunning*? 

> Hyperparameter tunning adalah sebuah aktivitas untuk menemukan hyperparamter yang paling tepat untuk menghasilkan model yang baik

Seperti yang ditelah dijelaskan sebelumnya, proses hyperparamter tunning sangat bergantung dengan algoritma atau model yang digunakan. Oleh karena itu, teknik ini juga sering disebut sebagai teknik tunning ***model centric***. Dalam praktiknya, *model centric tunning* tidak selalu menghasilkan model yang baik. Hal ini karena pada *model centric tunning*, kita hanya berfokus pada konfigurasi hyperparameter. Padahal, kebanyakan permasalahan dari pembuatan model machine learning adalah pada data. Oleh karena itu, terdapat pendekatan lain berfokus pada data untuk menghasilkan model yang baik. Pendekatan tersebut dikenal dengan istilah ***prioritize data centric tunning***. Berbeda dengan *model centric*, pada *priortize data centric*, kita akan berfokus pada datanya, seperti melakukan pemilihan sampling yang baik, *cleansing*, *augmenting*, ataupun memodifikasi data untuk menghasilkan model yang baik. Mengapa ada pendekatan *prioritize model centric*? Hal ini dikarenakan terdapat fakta bahwa, permasalah utama dalam membuat model machine learning biasanya diawali dengan data yang tidak bersih atau data kurang baik.

### Parameter vs. hyperparamter

Kita telah belajar tentang hyperparamter secara umum, lalu apa perbedaan parameter dan hyperparamter pada sebuah algoritma machine learning? Perbedaan mendasarnya adalah sebagai berikut,

> Parameter adalah nilai-nilai yang dihasilkan oleh algoritma machine learning pada saat proses "pembelajaran" berlangsung. Dengan kata lain, nilai yang dihasilkan merupakan nilai berdasarkan data yang diinputkan.

Sebagai contoh adalah nilai koenfisien $(\beta_0, \beta_1, \beta_2, ..., \beta_n)$ pada regresi linier, atau nilai bobot dan bias pada NN. Nilai nilai tersebut akan adaptif sesuai dengan data yang diinputkan, **tanpa harus didefinisikan oleh pengembang model machine learning**.

Disisi lain, 

> Hyperparameter murapakan nilai yang diinputkan oleh pengembang model dikarenakan nilai-nilai ini tidak bisa didapatkan dari data yang diinputkan kedalam model.

Sebagai contoh adalah nilai $(\alpha)$ atau learning rate, epoch, dan batch pada NN. Contoh lain adalah jumlah tree pada random forest.

### Mencari nilai hyperparameter

Setelah mengatahui apa itu hyperparameter, selanjutnya pasti akan muncul sebuah pertanyaan tentang "bagaimana mencari nilai hyperparameter yang tepat?". Pertanyaan ini wajar ditanyakan karena sifat nilai hyperparameter yang harus diinputkan oleh pengembang sebuah model machine learning. Secara umum, terdapat 3 teknik yang sering digunakan untuk mencari nilai hyperparameter, yaitu,

1. manual search
2. grid search
3. random search

Ketiga teknik pencarian ini masuk kedalam kategori pencarian hyperparameter dengan istilah *exhausive search*. Dikatakan sebagai pendekatan *exhausive search* dikarenakan teknik yang digunakan adalah pencarian secara mendalam (atau mencari seluruh kemungkinan yang ada).

> Manual search melakukan pencarian nilai hyperparameter secara manual. **LITERALY MANUAL**! Pengembang model berhak menentukan nilai hyperparameter tanpa ada aturan tertentu (alias coba-coba). Cara ini merupakan cara yang paling *straighforward* dalam teknik *exhausive search*.

> Grid search melalukan pencarian nilai hyperparameter untuk semua kemungkinan yang telah ditentukan oleh pengembang model. Setiap kombinasi dari hyperparameter yang ditentukan oleh pengembang model akan dievaluasi.

Sebagai contoh, kita akan melakukan tunning pada decision tree pada hyperparameter kriteria dan level dari tree. Kita telah menentukan kriteria yang akan dievaluasi adalah 'gini', 'entropy', dan 'log_loss'. Sedangkan kedalaman tree kita akan mencoba mulai dari 5 hingga 10. Maka, pada grid search, setiap kombinasi kriteria dan kedalaman akan dievlauasi satu per satu.

> Random search melakukan pencarian nilai hyperparamter dengan teknik yang hampir sama dengan grid search. Perbedaan utama random search dengan grid search adalah pada random search, kombinasi dari setiap hyperparamete akan dicari secaran acak. Untuk nilai hyperparamter yang telah dipilih pada iterasi sebelumnya, tidak akan mempengaruhi nilai hyperparameter yang dipilih untuk iterasi selanjutnya.